In [ ]:
# utils
# 1. .txt 파일 읽어들인다.
# 2. 전처리 파일이 있으면 전처리된 파일 로드, 없으면 전처리후 파일 저장
#
# 전처리 파일
# vocab: vocabulary dic {'char': idx}
# tensor: 읽어들인 파일의 문자들을 vocab 기준으로 숫자로 변환한 배열. (None, ) size.

In [254]:
import numpy as np
import collections

import os
import pickle

import tensorflow as tf

In [278]:
class TextLoader():
    def __init__(self, src_dir, batch_size, seq_length):
        self.vocab = None
        self.vocab_size = None
        self.tensor = None
        self.batch_size = batch_size
        self.seq_length = seq_length
        
        shake = 'tinyshakespeare/input.txt'
        sample = 'sample.txt'
        self.src_path = os.path.join(src_dir, sample)
        self.vocab_path = os.path.join(src_dir, 'vocab.pkl')
        self.tensor_path = os.path.join(src_dir, 'tensor.npy')
        
        if os.path.exists(self.vocab_path) and os.path.exists(self.tensor_path):
            print('전처리된 파일 로드중...')
            self.load_preprocessed()
        else:
            print('데이터 전처리중...')
            self.preprocess()
            
        self.create_batch()
        self.reset_batch_pointer()
        print('배치 생성 완료')
        
    def preprocess(self):
        # 파일 읽기
        with open(self.src_path, 'r') as fp:
            data = fp.read()
        print(data)
        self.chars = sorted(set(data))                           # ['a','b','c'...]
        self.vocab_size = len(chars)                             # vocab size
        self.vocab = {j:i for i, j in enumerate(self.chars)}     # vocab dic {'char': idx}
        self.tensor = np.array(list(map(self.vocab.get, data)))  # tensor ('char'->idx)
        # 파일 저장
        with open(self.vocab_path, 'wb') as fp:
            pickle.dump(self.vocab, fp)
        np.save(self.tensor_path, self.tensor)
            
    def load_preprocessed(self):
        with open(self.vocab_path, 'rb') as fp:
            self.vocab = pickle.load(fp)
        self.vocab_size = len(self.vocab)
        self.chars = sorted(self.vocab.keys())
        self.tensor = np.load(self.tensor_path)
        
    def create_batch(self):
        self.total_batch = self.tensor.size//(self.batch_size*self.seq_length)
        if self.total_batch == 0:
            assert False, "Not enough data. Make seq_length and batch_size small."
        self.tensor = self.tensor[:self.batch_size*self.seq_length*self.total_batch]
        xdata = self.tensor
        ydata = np.copy(self.tensor)
        # ydata 를 xdata 를 한칸 왼쪽으로 쉬프트한 형태로 구성.
        # b  c  d  a (ydata)
        # ==rnn cell==
        # a  b  c  d (xdata)
        ydata[:-1] = xdata[1:]
        ydata[-1] = xdata[0]
        
        self.x_batches =  np.split(xdata.reshape(self.batch_size, -1), self.total_batch, 1)
        self.y_batches = np.split(ydata.reshape(self.batch_size, -1), self.total_batch, 1)
        
    # 배치 불러오고 포인터를 1만큼 증가.
    def next_batch(self):
        x, y = self.x_batches[self.pointer], self.y_batches[self.pointer]
        self.pointer += 1
        return x,y
    
    def reset_batch_pointer(self):
        self.pointer = 0
    
    def convert_idx2char(self, num):
        char = self.chars[num]
        return char

    
    
data_dir = 'data'
batch_size = 16
seq_length = 16
hidden_size = 64
learning_rate = 0.05
num_epoch = 2
num_hidden_layers = 2
grad_clip = 5

data_loader = TextLoader(data_dir, batch_size, seq_length)

chars = data_loader.chars
vocab = data_loader.vocab
vocab_size = data_loader.vocab_size

전처리된 파일 로드중...
배치 생성 완료


In [280]:
# graph

# reset graph
tf.reset_default_graph()

# 인풋/타겟 데이터, 배치 사이즈를 입력받기 위한 플레이스홀더
input_data = tf.placeholder(tf.int32, shape=[None, None])  # input: [batch_size, seq_length]
target_data = tf.placeholder(tf.int32, shape=[None, None]) # target: [batch_size, seq_length]
state_batch_size = tf.placeholder(tf.int32, shape=[])

# RNN 마지막 히든레이어 출력을 소프트맥스 출력값으로 변환해주기 위한 변수
softmax_w = tf.Variable(tf.random_normal(shape=[hidden_size, vocab_size]), dtype=tf.float32)
softmax_b = tf.Variable(tf.random_normal(shape=[vocab_size]), dtype=tf.float32)

# 히든레이어 수 만큼 LSTM cell(히든레이어) 선언
cells = []
for _ in range(num_hidden_layers):
    cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
    cells.append(cell)
    
# cell을 종합해서 RNN을 정의
cell = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)

# 인풋데이터를 변환하기 위한 임베딩 매트릭스 선언
# vocab_size -> hidden_size
embedding = tf.Variable(tf.random_normal(shape=[vocab_size, hidden_size]), dtype=tf.float32)
inputs = tf.nn.embedding_lookup(embedding, input_data)

# 초기 state 값을 0으로 초기화
initial_state = cell.zero_state(state_batch_size, tf.float32)

# 학습을 위한 tf.nn.dynamic_rnn을 선언
# outputs: [batch_size, seq_length, hidden_size]
outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state, dtype=tf.float32)

# output을 [batch_size * seq_length, hidden_size] shape으로 바꿈
output = tf.reshape(outputs, [-1, hidden_size])

# 최종 출력값을 설정
# logits: [batch_size * seq_length, vocab_size]
# softmax 를 적용하기 위해 vocab_size 로 shape 을 바꾼다.
# output.shape: (?,64)
# logits.shape: (?,36)
logits = tf.matmul(output, softmax_w) + softmax_b
probs = tf.nn.softmax(logits)

# 크로스 엔트로피 손실함수 정의
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=target_data))

# 옵티마이저 선언하고 옵티마이저에 Gradient Clipping을 적용
# grad_clip 보다 큰 Gradient를 5로 Clipping 한다
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.apply_gradients(zip(grads, tvars))

# 세션을 열고 학습 진행
with tf.Session() as sess:
    pass

In [266]:
1e-3

0.001